In [196]:
import numpy as np

Generador de enteros de 32 bits de periodo 2^128 - 1.

In [197]:
class XORSHIFT128:
    def __init__(self, x, y, z, w):
        self.periodo = 2**128 - 1
        self.x = x
        self.y = y
        self.z = z
        self.w = w

    def rnd(self):
        tmp = (self.x^(self.x<<11))
        self.x=self.y
        self.y=self.z
        self.z=self.w
        self.w = (self.w ^ (self.w >> 19)) ^ (tmp ^ (tmp >> 8))
        return self.w & 0xFFFFFFFF # Salida de 32 bits
    def rnd_cero_uno(self):
        return self.rnd()/self.periodo

generador = XORSHIFT128(123456789, 362436069,521288629,88675123)
for _ in range(5):
    numero_generado = generador.rnd()
    print(numero_generado)

3869459946
2371244454
4211465258
2627033330
1877824159


Para generar números aleatorios entre 0 y 1 debemos dividir entre el periodo. Para resolver este ítem se implementó el método rnd_cero_uno()

In [198]:
for _ in range(5):
    numero_generado = generador.rnd_cero_uno()
    print(numero_generado)

7.437632728080884e-30
4.397596095091465e-30
6.455106034073022e-31
5.422140029455839e-30
9.306679557497613e-30
